# ⭐ radix sort를 구현해보자.

## 고정된 3자리 정수 라면

In [8]:
arr = [329, 457, 657, 839, 436, 720, 355]

### **1의 자리 정렬**
#### > 주의 사항
 - bucket을 만들때, 자릿수별로 만들어야 하니 미리 0~9 까지 만들어 놓고, 해당 자리수에 넣어야 하기 때문에 이중리스트로
 - 이중리스트를 풀어서 원소만 넣을것이기 때문에 그대로 추가하는 append 대신 extend로 풀어서 넣기

In [9]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = x%10
  bucket[digit].append(x)

arr = []

for b in bucket:
  arr.extend(b)

print(arr)

[720, 355, 436, 457, 657, 329, 839]


### 10의 자리 정렬

In [10]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = (x//10)%10
  bucket[digit].append(x)

arr = []
for b in bucket:
  arr.extend(b)

print(arr)

[720, 329, 436, 839, 355, 457, 657]


### 100의 자리 정렬

In [11]:
bucket = [[] for _ in range(10)]

for x in arr:
  digit = (x//100)%10
  bucket[digit].append(x)

arr = []

for b in bucket:
  arr.extend(b)

print(arr)

[329, 355, 436, 457, 657, 720, 839]


## 가변길이 정수라면

In [12]:
arr = [38295, 291, 692602, 293, 291, 12, 0, 12985]

In [13]:
max_len = len(str(max(arr)))

In [14]:
for i in range(max_len):
  bucket = [[] for _ in range(10)]
  div = 10**i
  for x in arr:
    digit = (x//div) %10
    bucket[digit].append(x)

  arr = []

  for b in bucket:
    arr.extend(b)

In [15]:
print(arr)

[0, 12, 291, 291, 293, 12985, 38295, 692602]


## ❤ Radix Sort 구현하며 겪은 핵심 실수 정리

---

## 1. bucket 구조 오해

❌ 잘못된 생각
`bucket = []`
임시 저장소 하나만 있으면 된다고 생각함.

⭕ 올바른 구조
`bucket = [[] for _ in range(10)]`
→ 0~9 자릿수별 분류함 10개가 필요함.

---


## 2. append 와 extend 혼동
❌ 실수
`arr.append(b)`
→ 리스트 안에 리스트가 들어가 구조 붕괴.

⭕ 정답
`arr.extend(b)`
→ 버킷 안의 값들을 풀어서 삽입.

---

## 3. bucket 초기화 위치 오류
❌ 실수
`bucket = [[] for _ in range(10)]   # for 밖`
→ 이전 단계 데이터가 계속 누적되어 값 폭증.

⭕ 정답
```python
for exp in range(max_len):
    bucket = [[] for _ in range(10)]   # 반드시 for 안
```

---

## 4. 1의 자리 정렬 생략
❌ 실수
`for i in range(1, max_len):`
→ 10의 자리부터 시작 → 안정 정렬 깨짐 → 중복 순서 붕괴.

⭕ 정답
`for exp in range(max_len):`

---


## 5. 내림차순 구현 착각
❌ 실수
`for b in reversed(bucket):`
→ LSD 방식에서 안정성 파괴.

⭕ 정답
오름차순으로 완성 후
`arr.reverse()`

---


## 6. arr = bucket 대입 실수
❌ 실수
`arr = bucket`
→ 2차원 구조로 변해 다음 단계에서 에러.

⭕ 정답
```python
arr = []
for b in bucket:
    arr.extend(b)
```

# ⭐ Floyd-Warshall 을 구현해보자.

## ▶ **Floyd-Warshall 알고리즘** ?

모든 정점 쌍 사이의 최단 거리를 구하는 알고리즘

> 언제써?
> - "A에서 B까지 최단 거리”
> - "B에서 C까지 최단 거리”
> - 이런 걸 모든 쌍에 대해 전부 알고 싶을 때 사용한다.


## ▶ 이차원 리스트의 의미

`dist[i][j]` : `i`번 정점에서 `j`번 정점까지 가는 **현재까지 알고 있는 최단 거리**

> 초기 상태는
> - 자기 자신 -> `0`
> - 간언이 있으면 -> 그 가중치
> - 없으면 -> 매우 큰 값($\infty$)

## ▶ 핵심 아이디어

> `i -> j`로 가는것보다 `i -> k -> j`로 우회하는게 더 짧으면 바꾼다.

이걸 **모든 `i` 모든 `j` 모든 `k`** 조합에 대해 반복

In [16]:
INF = int(1e9)
n = 3
m = 2

In [17]:
# graph 없을때 n, m 입력받기용
# graph = [[INF]*(n+1) for _ in range(n+1)]

# 자기 자신으로 가는 경우를 0 으로 초기화
# for a in range(len(graph)):
#   for b in range(len(graph)):
#     if a == b:
#       graph[a][b] = 0

# 초기 데이터
graph = [
    [0,   3,  float('inf'), 7],
    [8,   0,  2,            float('inf')],
    [5,   float('inf'), 0,  1],
    [2,   float('inf'), float('inf'), 0]
]

In [18]:
%%time
for i in range(len(graph)):
  for j in range(len(graph)):
    for k in range(len(graph)):
      graph[i][j] = min(graph[i][j], graph[i][k] + graph[k][j])

CPU times: user 43 µs, sys: 4 µs, total: 47 µs
Wall time: 51.7 µs


In [19]:
graph

[[0, 3, 5, 6], [7, 0, 2, 3], [3, 6, 0, 1], [2, 5, 7, 0]]

## ❤ 피드백

> k번째 반복이 끝났을 때 `graph[i][j]` 는 중간 노드로 `0 ~ k`까지만 사용하는 단거리 여야 한다.

이 조건을 만족하려면
k가 가장 바깥 루프에 있어야 한다.

In [20]:
n = len(graph)

In [21]:
%%time
for k in range(n):          # 거쳐가는 노드
    for i in range(n):      # 출발 노드
        for j in range(n):  # 도착 노드
            if graph[i][j] > graph[i][k] + graph[k][j]:
                graph[i][j] = graph[i][k] + graph[k][j]


CPU times: user 24 µs, sys: 3 µs, total: 27 µs
Wall time: 29.3 µs


In [22]:
graph

[[0, 3, 5, 6], [5, 0, 2, 3], [3, 6, 0, 1], [2, 5, 7, 0]]

# ⭐ Gradient Descent(경사하강법)을 하드코딩 해보자!

---

## ▶ Gradient Descnet?
> 함수의 기울기(미분값)을 이용하여 함수값을 가장 빠르게 감소시키는 방향으로 조금씩 이동하면서 최소값을 찾는 최적화 알고리즘

---

## ▶ 왜 **기울기 반대 방향**?
> 기울기는 현재 위치에서 **함수값이 가장 빠르게 증가하는 방향**이기 때문에

따라서 `- 기울기 방향`은 함수값이 *가장 빠르게 감소하는 방향* 이 된다.

---

## ▶ 수식으로 정리해볼까?

- 단변수
$$ x_{t+1}​=x_t​−α⋅f'(x_t​) $$

- 다변수
$$ W_{t+1} = W_t - α⋅\nabla f(W_t) $$

|기호|의미|
|---|---|
|$~~~~~~~~α~~~~~~~~$| 학습률 (step size)|
|$$ f'(x)  $$ | 기울기|
|$$ \nabla f$$ | 모든 변수에 대한 기울기 벡터|

---

## ▶ 학습률 ( $α~$)
학습률은 **한 번 이동할 때 얼마나 크게 이동할지 결정하는 값**이다.
|크기|현상|
|---|---|
|너무 큼|최소값을 넘어 튕기며 발산|
|적당함|안정적으로 최소값 수렴|
|너무 작음|수렴은 하지만 매우 느림|

---

## ▶ 정답을 찾지 못하는 이유

### (1) 지역 최솟값 문제
경사하강법은 출발 위치 근처의 지역 최솟값(Local Minimum)에 빠질 수 있다.

### (2) 평평한 구간 문제 (Plateau)
기울기가 거의 $0$인 평평한 영역에서는 이동 속도가 극도로 느려진다.

### (3) 안장점 (Saddle Point)
기울기는 $0$이지만 최소도 최대도 아닌 지점에서 멈춰버릴 수 있다.

---

## ▶ Gradient Descent 종류
|종류|설명|
|---|---|
|batch GD| 전체 데이터로 한 번에 기울기 계산|
|SGD| 데이터 하나마다 기울기 계산|
|Mini-batch GD| 일부 묶음 단위로 계산|



## ❤ 구현 시 주의사항

---

### ① 학습률 범위

만약 $ f(x) = 3x^2 + 0.1x $ , $ f'(x) = 6x + 0.1 $

```python
def function_1(x):
  return 3 * x**2 + 0.1 * x
```

초기값 $ x=2$ 이면 $ f'(2) = 12.1$

경사하강 $1$ 스텝 : $ x_{new} = 2-10⋅12.1 = -119 $

첫 스텝에서 기울기의 10배를 넘겨버림 -> 발산 -> 미쳐 날뛰기 시작.

> 이 예시의 경우 **이차함수(볼록 함수)** 라서
> 안정적인 α는 보통 `0.01 ~ 0.001` 수준으로 잡는다.

---

### ② 수치미분 h의 값

$ h=1$ 은 너무 크다.
```python
def diff(f,x):
  h = 1
  return (f(x+h)-f(x-h)) / h
```
미분 수준이 아니라 흉내 낸거라 기울기가 계속 **과대추정**된다.

---

### ③ 발산

$ x_{t+1} = x_t - 10⋅(6x_t + 0.1)  $  
➡ $~ x_{t+1} = -59x_t -1 $  
➡  $~ |-59|>1$

> 수열이 **지수적으로 폭발하도록 설계**된 수식

---

### ④ 진짜 최솟값은?

$ f'(x) = 6x+0.1 = 0 $ ➡ $ x= -0.016666~ ... $

---

### ⑤ 정리

|항목|현재 상태|문제|
|---|---|---|
| 학습률|$10$|너무 큼 -> 발산|
|수리미분 $h$| $1$ | 기울기 과대
|함수 형태| 2차 함수| 작은 학습률 필수|

## 컨셉 1

```
초기값 x 를 하나 정한다
학습률 alpha 를 정한다
반복 횟수 max_iter 를 정한다

for 반복횟수 만큼:
    현재 기울기 = f'(x) 계산
    x = x - alpha * 현재 기울기
    x 값 기록 (필요하면)

```

In [23]:
def diff(f,x):
  h = 1
  return (f(x+h)-f(x-h)) / h

In [24]:
def function_1(x):
  return 3 * x**2 + 0.1 * x

In [25]:
diff(function_1, 2)

24.2

In [26]:
x = 2
alpha = 10
max_iter = 5
answer = []

In [27]:
for _ in range(max_iter):
  diff_value = diff(function_1, x)
  x = x - alpha * diff_value
  answer.append(x)

In [28]:
answer

[-240.0,
 28557.999999999884,
 -3398404.000002861,
 404410074.00780964,
 -48124798245.99219]

실패 한 것

---

In [29]:
def gd(f, grad_f, x, alpha, max_iter):
  answer = []

  for _ in range(max_iter):
    g = grad_f(x)
    x = x - alpha * g
    answer.append(x)

  return x, answer

In [30]:
def f1(x):
  return 3*x**2 + 0.1*x

In [31]:
def grad_f1(x):
  return 6*x+0.1

In [32]:
x0=2
x_min, trace = gd(f1, grad_f1, x0, alpha=0.01, max_iter=100)

In [33]:
x_min

-0.012522669212777409

## 컨셉 2

```
초기값 x 설정
학습률 alpha 설정
허용 오차 epsilon 설정

while True:
    현재 기울기 = f'(x)
    
    만약 |현재 기울기| < epsilon 이면:
        break

    x = x - alpha * 현재 기울기
```

In [34]:
def gradient_descent_until(f, grad_f, x, alpha=0.01, epsilon=1e-6):
    history = []

    while True:
        g = grad_f(x)

        if abs(g) < epsilon:
            break

        x = x - alpha * g
        history.append(x)

    return x, history

In [35]:
x0 = 2
x_min, trace = gradient_descent_until(f1, grad_f1, x0)

In [36]:
x_min

-0.016666504340897184

## 컨셉 3 다변수

```
초기 벡터 W 설정
학습률 alpha 설정

for 반복:
    gradient = 모든 변수에 대한 편미분 벡터
    W = W - alpha * gradient
```

In [37]:
pip install numpy

In [38]:
import numpy as np

In [39]:
def gradient_descent_multi(grad_f, W, alpha=0.01, max_iter=100):
    history = []

    for _ in range(max_iter):
        grad = grad_f(W)
        W = W - alpha * grad
        history.append(W.copy())

    return W, history

In [40]:
# f(x, y) = x^2 + y^2
def grad_f2(W):
    x, y = W
    return np.array([2*x, 2*y])

In [41]:
W0 = np.array([5.0, -3.0])
W_min, trace = gradient_descent_multi(grad_f2, W0)

In [42]:
W_min

array([ 0.66309778, -0.39785867])

## 결과

- 단변수는 $x≈-0.01666~...$ 근처로 수렴
- 다변수는 `[0, 0]`으로 수렴

# ⭐ Back Propagation(역전파)를 구현해보자.

## ▶ Back Propagation?

> 출력 오차를 기준으로 **연쇄법칙(Chain Rule)**을 이용해 각 가중치의 기울기를 **뒤에서 부터** 계산하는 알고리즘.

## ▶ Back Propagation이 왜 필요한가?

경사하강법의 문제
- `loss`는 출력층에서만 계산됨
- 가중치는 `입력층 ~ 은닉층 ~ 출력층` 전체에 흩어져 있음

> "이 오차의 책임이 각 가중치에 얼마나 있나?" 계산해야 하기 때문에 이를 해결하는 Back Propagation이 필요

## ▶ 최소 신경망 구조
```
입력 x
 ↓
가중치 w1
 ↓
은닉층 (선형)
 ↓
활성함수
 ↓
가중치 w2
 ↓
출력 ŷ
 ↓
손실 L
```
↪ 추후 레이어를 늘리는 것으로 발전

## ▶ 수식 흐름 (Foward -> Backward)

- Forward Propagation
$$
z_1 = w_1x \\
a_1 = f(z_1)  \\
z_2 = w_2 a_1 \\
\widehat{y} = z_2
$$

- Loss
$$
L = \frac{1}{2} (\widehat{y} - y )^2
$$

## ▶ Back Propagation 핵심 (연쇄법칙)
- 츨력층
$$
\frac{\partial L}{\partial \widehat{y}} = \widehat{y} - y \\
\frac{\partial L}{\partial w_2} = \frac{\partial L}{\partial \widehat{y}} ⋅
\frac{\partial \widehat{y}}{\partial z_2} ⋅
\frac{\partial z_2}{\partial w_2}
$$

- 은닉층
$$
\frac{\partial L}{\partial w_1} =
\frac{\partial L}{\partial \widehat{y}} ⋅
\frac{\partial \widehat{y}}{\partial z_2} ⋅
\frac{\partial z_2}{\partial a_1} ⋅
\frac{\partial a_1}{\partial z_1} ⋅
\frac{\partial z_1}{\partial w_1}
$$

> 이게 Back Propagation (출력 -> 입력 방향)

## ❤ 주의사항

### 순서 🌟🌟🌟

1. Forward 계산 함수
```python
z1 = w1 * x
   = 0.5 * 1.0
   = 0.5

a1 = ReLU(z1)
   = 0.5

z2 = w2 * a1
   = 1.0 * 0.5
   = 0.5

y_hat = z2

```
2. Loss 계산
```python
L = 1/2 * (y_hat - y)^2
  = 1/2 * (0.5 - 2.0)^2
  = 1.125
```
3. 출력층 gradient
4. 은닉층 gradient
5. 가중치 업데이트

### 전체수도코드
```python
초기값 x, y 설정
초기 가중치 w1, w2 설정
학습률 alpha 설정

for 반복 횟수 만큼:

    # Forward
    z1 = w1 * x
    a1 = ReLU(z1)
    z2 = w2 * a1
    y_hat = z2

    # Loss
    L = 0.5 * (y_hat - y)^2

    # Backward
    dL_dyhat = y_hat - y
    dL_dw2 = dL_dyhat * a1
    dL_da1 = dL_dyhat * w2
    dL_dz1 = dL_da1 * ReLU'(z1)
    dL_dw1 = dL_dz1 * x

    # Update
    w2 = w2 - alpha * dL_dw2
    w1 = w1 - alpha * dL_dw1
```

### 참고 (ReLU 함수)

- ReLU : 0 보다 작은 값이면 return 0, 아니면 return x
``` python
def ReLU_value(x):
  return max(0, x)
```
- ReLU' : 0 보다 작으면 0, 아니면 1
```python
def ReLU_diff(z):
  return 1 if z>0 else 0
```

### 참고 (Sigmoid 함수)

- Sigmoid : 입력을 $(0,~1)$범위의 확률로 변환
$$
σ(z) = \frac{1}{1 + e^{-z}}
$$

```python
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))
```

- Sigmoid` : Sigmoid의 미분은 자기 자신으로 표현 가능

$$
σ'(z) = σ(z)(1-σ(z))
$$

```python
def sigmoid_diff(z):
    s = sigmoid(z)
    return s * (1 - s)
```

In [43]:
# 초기값 설정

# 입력
x = 1.0
y = 2.0

# 가중치
w1 = 0.5
w2 = 1.0

# 학습률(alpha)
alpha = 0.1

In [44]:
def ReLU(z1):
  return max(0, z1)

In [45]:
def de_ReLU(z1):
  return 1 if z1>0 else 0

In [46]:
echo = 10

for step in range(echo):
  # 1. Forward 계산 함수
  z1 = w1 * x
  a1 = ReLU(z1)
  z2 = w2*a1
  y_hat = z2

  # 2. Loss
  L = 0.5 * ((y_hat-y)**2)

  # 3. Backward
  dL_dyhat = y_hat - y
  dL_dw2 = dL_dyhat * a1
  dL_da1 = dL_dyhat * w2
  relu_grad = de_ReLU(z1)
  dL_dz1 = dL_da1 * relu_grad
  dL_dw1 = dL_dz1 * x

  # 5. Update
  w2 = w2 - alpha * dL_dw2
  w1 = w1 - alpha * dL_dw1

  # 6. print
  print(f"{step} : ", L, w1, w2)

0 :  1.125 0.65 1.075
1 :  0.84662578125 0.789884375 1.15958125
2 :  0.5875983418702183 0.9155905069162853 1.2452098417372004
3 :  0.3697120184502426 1.022665813537232 1.3239412579059942
4 :  0.20869064779657115 1.108199109521525 1.3900106376461856
5 :  0.10561215005902769 1.1720828098484195 1.4409425211137026
6 :  0.048390373370087676 1.2169099612144347 1.4774055533031967
7 :  0.02042836252784135 1.2467728284265072 1.5020030109880853
8 :  0.008108178107618139 1.2658998541137958 1.517879847280071
9 :  0.0030823907670939578 1.2778176581570866 1.5278192021155628


In [47]:
z1, a1, dL_dw1

(1.2658998541137958, 1.2658998541137958, -0.1191780404329086)

## 은닉층 노드 여러 개로 확장

### 기존

```css
x → w1 → ReLU → w2 → ŷ
```

### 확장 (은닉 2개)
```css
          w1_1
        ↗
x ────┤       → ReLU → a1 ─┐
        ↘                         ├→ ŷ
          w1_2        ReLU → a2 ─┘
```
- 은닉층이 **스칼라 -> 벡터**
- 가중치가 **하나 -> 배열**

### 변화

```python
# 입력
x : 스칼라 #동일
y : 스칼라 #동일

# 가중치
w1 : 하나 -> 배열
w2 : 하나 -> 배열

# 은닉층
z1 : 스칼라 -> 벡터 => z1_i = w1_i *x
a1 : 스칼라 -> 벡터 => a1_i = ReLU(z1_i)

# 출력층
y_hat = w2_1 * a1_1 + w2_2 * a1_2

# Loss
L = 0.5 * (y_hat - y)^2
```

### 중요

> 출력 오차 하나가 은닉 노드 개수만큼 분기되어 전달됨

### 흐름

```python
# 출력층 가중치
dL/dw2_i = (y_hat - y) * a1_i

# 은닉층 가중치
dL/dw1_i = (y_hat - y) * w2_i * ReLU'(z1_i) * x
```


In [48]:
# 초기값 설정
import numpy as np

# 입력
x = 1.0
y = 2.0

# 가중치
np.random.seed(10)
w1 = np.random.rand(3)
w2 = np.random.rand(3)

# 학습률(alpha)
alpha = 0.1

n_hidden = len(w1)
z1 = [0] * n_hidden
z2 = [0] * n_hidden
a1 = [0] * n_hidden
dL_dw1 = [0] * n_hidden
dL_dw2 = [0] * n_hidden

In [49]:
def ReLU(z1):
  return max(0, z1)

In [50]:
def de_ReLU(z1):
  return 1 if z1>0 else 0

In [51]:
echo = 10

for _ in range(echo):

  # Forward (은닉)
  for i in range(n_hidden):
    z1[i] = w1[i] * x
    a1[i] = ReLU(z1[i])
    z2[i] = w2[i] * a1[i]

  # Forward (출력) y_hat = w2_1 * a1_1 + w2_2 * a1_2
  y_hat = 0
  for i in range(n_hidden):
    y_hat += w2[i] * a1[i]

  # Loss L = 0.5 * (y_hat - y)^2
  L = 0.5 * (y_hat -y)**2

  # Backward (출력)
  dL_dyhat = y_hat - y

  # 출력층 가중치
  for i in range(n_hidden):
    dL_dw2[i] = dL_dyhat * a1[i]

  # 은닉층 가중치
  for i in range(n_hidden):
    dL_dw1[i] = dL_dyhat * w2[i] * de_ReLU(z1[i]) * x

  # Update
  for i in range(n_hidden):
    w2[i] -= alpha * dL_dw2[i]
    w1[i] -= alpha * dL_dw1[i]

In [52]:
print(z1)
print(a1)
print(dL_dw1)
print(dL_dw2)

[np.float64(1.1882212968675692), np.float64(0.2556159634628483), np.float64(0.7936349207977484)]
[np.float64(1.1882212968675692), np.float64(0.2556159634628483), np.float64(0.7936349207977484)]
[np.float64(-0.04083661337804628), np.float64(-0.019175072715071757), np.float64(-0.018796957944251874)]
[np.float64(-0.041321387346985206), np.float64(-0.008889258479179056), np.float64(-0.027599316777801104)]


In [53]:
print(np.array(w1, dtype=float))

[1.19230496 0.25753347 0.79551462]


### 확장 : 벡터화 + Bias
<br></br>

✈ 벡터화에서 중요한건 `@`연산

> `*`은 원소별 곱 (Element-wise)<br>
> `@`는 행렬 곱 (선형대수)

```python
# * 연산
A * B -> [1 2]   *   [3 4]  =  [3 8]

# @ 연산
A @ B -> [1 2]   @   [3]
                     [4]   =  [11]
```


<br></br>
✈  **bias**는
> "입력이 $0$이어도 누련이 활성화될 수 있게 하는 이동값"

$$
z = Wx + b \\
~\\
W : 기울기(회전) \\
b : 절편(평행이동)
$$

<br></br>
✈ **bias**가 없다면

`z = W * x`
- 입력이 $0$이면 -> 출력도 무조건 $0$
- 결정경계가 **항상 원점을 지나야 함**

-> 표현하는데 제한이 생김

<br></br>
✈ Back Propagation에서 bias 가 어떻게 학습이 되나?

```python
# 은닉층
dL_db1 = dL_dz1

# 출력층
dL_db2 = dL_dyhat
```

- bias는 입력과 무관하고
- 그래서 미분 시 그대로 gradient가 전달

In [54]:
import numpy as np

np.random.seed(10)

# 입력
x = np.array([1.0, 2.0])
y = 2.0

# 가중치 + bias
W1 = np.random.rand(3, 2)
b1 = np.random.rand(3)

W2 = np.random.rand(1, 3)
b2 = np.random.rand(1)

alpha = 0.1

In [55]:
def relu(z):
    return np.maximum(0, z)

def relu_grad(z):
    return (z > 0).astype(float)

In [56]:
# Forward
z1 = W1 @ x + b1        # (3,)
a1 = relu(z1)           # (3,)

z2 = W2 @ a1 + b2       # (1,)
y_hat = z2[0]

# Loss
L = 0.5 * (y_hat - y) ** 2

# Back Propagation
dL_dyhat = y_hat - y        # scalar

dL_dW2 = dL_dyhat * a1.reshape(1, -1)   # (1,3)
dL_db2 = dL_dyhat

# Hidden
dL_da1 = dL_dyhat * W2.reshape(-1)       # (3,)
dL_dz1 = dL_da1 * relu_grad(z1)           # (3,)

dL_dW1 = dL_dz1.reshape(-1, 1) @ x.reshape(1, -1)   # (3,2)
dL_db1 = dL_dz1

#Update
W2 -= alpha * dL_dW2
b2 -= alpha * dL_db2

W1 -= alpha * dL_dW1
b1 -= alpha * dL_db1

In [57]:
for step in range(100):
    z1 = W1 @ x + b1
    a1 = relu(z1)

    y_hat = (W2 @ a1 + b2)[0]
    L = 0.5 * (y_hat - y) ** 2

    dL_dyhat = y_hat - y

    dL_dW2 = dL_dyhat * a1.reshape(1, -1)
    dL_db2 = dL_dyhat

    dL_da1 = dL_dyhat * W2.reshape(-1)
    dL_dz1 = dL_da1 * relu_grad(z1)

    dL_dW1 = dL_dz1.reshape(-1, 1) @ x.reshape(1, -1)
    dL_db1 = dL_dz1

    W2 -= alpha * dL_dW2
    b2 -= alpha * dL_db2
    W1 -= alpha * dL_dW1
    b1 -= alpha * dL_db1

    if step % 10 == 0:
        print(step, L)

0 0.3961111452373531
10 1.512809528434771e-07
20 9.694175666383829e-14
30 6.213979816703268e-20
40 4.001191258132809e-26
50 6.162975822039155e-31
60 0.0
70 0.0
80 0.0
90 0.0


## 손실 함수

<br></br>
### Loss Function ?

> 손실함수는 **모델의 출력이 정답과 얼마나 다른지**를 수치로 나타낸 함수 <br>
> 출력이 정답과 **가까울**수록 **작아짐** <br>
> 목표 = **손실 최소화**

<br></br>
### 종류

#### 1. 평균제곱오차 (MSE)

$$
L = \frac{1}{n} \sum ( y_{pred} - y_{true} )^2
$$

- 차이를 제곱 -> 오차가 크게 벌어지면 패널티 큼
- 회귀 문제의 기본

```python
# 예제 (넘파이 벡터화)

import numpy as np

y_true = np.array([1.0, 0.0, 1.0])
y_pred = np.array([0.8, 0.3, 0.4])

loss = np.mean((y_pred - y_true) ** 2)
```

<br></br>
#### 2. 이진 크로스 엔트로피 (Binary Cross Entropy)

$$
L = - \frac{1}{n} \sum \left( y ⋅\log (\hat{y}) + (1-y)⋅\log(1- \hat{y} \right)
$$

- 확률 예측을 강하게 평가
- 틀린 확률에 엄청난 패널티

```python
# 예제
eps = 1e-8  # log(0) 방지

loss = -np.mean(
    y_true * np.log(y_pred + eps) +
    (1 - y_true) * np.log(1 - y_pred + eps)
)
```

<br></br>
### 흐름

```
x
 ↓
W1 @ x + b1
 ↓
activation
 ↓
W2 @ h + b2
 ↓
y_pred
 ↓
LOSS ← 여기!
```

#### MSE

In [58]:
import numpy as np

# 1. 데이터 (Batch)
# N=4, D=2
X = np.array([
    [1.0, 2.0],
    [2.0, 1.0],
    [3.0, 1.0],
    [1.0, 3.0]
])
y = np.array([2.0, 2.5, 3.5, 2.2])   # (N,)

N, D = X.shape

# 2. 파라미터 초기화
np.random.seed(10)

W1 = np.random.rand(3, D)   # (3,2)
b1 = np.random.rand(3)      # (3,)

W2 = np.random.rand(1, 3)   # (1,3)
b2 = np.random.rand(1)      # (1,)

alpha = 0.1

# 3. 활성함수
def relu(z):
    return np.maximum(0, z)

def relu_grad(z):
    return (z > 0).astype(float)

# 4. Forward
z1 = X @ W1.T + b1          # (N,3)
a1 = relu(z1)

z2 = a1 @ W2.T + b2         # (N,1)
y_hat = z2.reshape(-1)     # (N,)

# 5. Loss (MSE)
L = np.mean((y_hat - y) ** 2)

# 6. Backpropagation
# dL/dy_hat
dL_dyhat = (2 / N) * (y_hat - y)     # (N,)

# ---- Output layer ----
dL_dz2 = dL_dyhat.reshape(-1, 1)     # (N,1)

dL_dW2 = dL_dz2.T @ a1               # (1,3)
dL_db2 = np.sum(dL_dz2, axis=0)      # (1,)

# ---- Hidden layer ----
dL_da1 = dL_dz2 @ W2                 # (N,3)
dL_dz1 = dL_da1 * relu_grad(z1)      # (N,3)

dL_dW1 = dL_dz1.T @ X                # (3,2)
dL_db1 = np.sum(dL_dz1, axis=0)      # (3,)

# 7. Update
W2 -= alpha * dL_dW2
b2 -= alpha * dL_db2

W1 -= alpha * dL_dW1
b1 -= alpha * dL_db1

# 8. 출력 확인
print("Loss:", L)
print("y_hat:", y_hat)


Loss: 1.407067653616083
y_hat: [3.14030595 3.38864164 4.36633027 3.8696589 ]


#### Binary Cross Entropy

In [60]:
import numpy as np

# 1. 데이터 (Batch)
X = np.array([
    [1.0, 2.0],
    [2.0, 1.0],
    [3.0, 1.0],
    [1.0, 3.0]
])                       # (N,2)

y = np.array([1, 1, 0, 0])   # (N,)

N, D = X.shape

# 2. 파라미터 초기화
np.random.seed(10)

W1 = np.random.rand(3, D)   # (3,2)
b1 = np.random.rand(3)      # (3,)

W2 = np.random.rand(1, 3)   # (1,3)
b2 = np.random.rand(1)      # (1,)

alpha = 0.1

# 3. 활성함수
def relu(z):
    return np.maximum(0, z)

def relu_grad(z):
    return (z > 0).astype(float)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 4. Forward
z1 = X @ W1.T + b1          # (N,3)
a1 = relu(z1)

z2 = a1 @ W2.T + b2         # (N,1)
y_hat = sigmoid(z2).reshape(-1)   # (N,)

# 5. Binary Cross Entropy
eps = 1e-8   # log 안정성
L = -np.mean(
    y * np.log(y_hat + eps) +
    (1 - y) * np.log(1 - y_hat + eps)
)

# 6. Backpropagation
# 핵심 결과: dL/dz2
dL_dz2 = (y_hat - y).reshape(-1, 1) / N   # (N,1)

# ---- Output layer ----
dL_dW2 = dL_dz2.T @ a1               # (1,3)
dL_db2 = np.sum(dL_dz2, axis=0)      # (1,)

# ---- Hidden layer ----
dL_da1 = dL_dz2 @ W2                 # (N,3)
dL_dz1 = dL_da1 * relu_grad(z1)      # (N,3)

dL_dW1 = dL_dz1.T @ X                # (3,2)
dL_db1 = np.sum(dL_dz1, axis=0)      # (3,)

# 7. Update
W2 -= alpha * dL_dW2
b2 -= alpha * dL_db2

W1 -= alpha * dL_dW1
b1 -= alpha * dL_db1

# 8. 출력
print("Loss (BCE):", L)
print("y_hat (prob):", y_hat)
print("pred:", (y_hat > 0.5).astype(int))


Loss (BCE): 2.086203340234188
y_hat (prob): [0.95852505 0.96734767 0.98746146 0.97956098]
pred: [1 1 1 1]
